# 0.

1. 0个GT，只有context features

In [1]:
%load_ext autoreload
%autoreload 2

# 1. Preparations
## 1.1  Import

In [22]:
import pandas as pd
import numpy as np
import math
import random
import torch.utils.data
from mydataset import *
import torch.nn as nn
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter
from visdom import Visdom
from graphviz import Digraph
from torchviz import make_dot
from torch.nn.utils.rnn import pad_sequence
import pygraphviz as pgv
import networkx as nx
import matplotlib.pyplot as plt
import geomloss
import time

import scipy
from torch.nn import KLDivLoss
from torch.autograd.gradcheck import gradcheck
from config import bcolors

from importlib import reload
import config
import loss

In [23]:
reload(config)  # 必须reload！！
reload(loss)    # 必须reload！！
from config import DefaultConfig
from loss import cal_metric
from loss import loss_fn_v2
opt = DefaultConfig()

# nums of Gaussian kernels
N_gaussians = opt.N_gaussians

# dataset划分
batch_size = opt.batch_size
train_pct = opt.train_pct
vali_pct = opt.vali_pct
test_pct = opt.test_pct

# train and optim.
learning_rate = opt.learning_rate
lr_for_mu = opt.lr_for_mu   # 给mu单独设置learning rate

total_train_step = 0
total_test_step = 0

MIN_LOSS = opt.MIN_LOSS
SAFETY = opt.SAFETY

# Range【一般用不到】
a = 0
b = 255

# Training data的粒度，画图会使用到
SCALE = opt.SCALE
# Target data是target_5时
TARGET = opt.TARGET

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## 1.2 the data path


In [24]:
# Training data
train_path = opt.train_path

# Target data
target_path_metric = opt.target_path_metric
target_path_loss = opt.target_path_loss

# data keys
data_key_path = opt.data_key_path

# NLL metric
NLL_metric_path = opt.NLL_metric_path

# Net path
net_root_path = opt.net_root_path

# 2. Dataset and Dataloader
1. DataLoader中的`shuffer=True`表示在每一次epoch中都打乱所有数据的顺序，然后以batch为单位从头到尾按顺序取用数据。这样的结果就是不同epoch中的数据都是乱序的,设置随机种子的作用就是让你的每一次训练都乱的一样，

## 2.1 Dataset and spliting


- 设置随机数种子

In [4]:
def setup_seed(seed):
    torch.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    # torch.backends.cudnn.deterministic = True

setup_seed(7)

In [25]:
dataset = myDataset(train_path, target_path_metric, target_path_loss, data_key_path, NLL_metric_path)

In [32]:
# 使用全部的data
# DATA_len = dataset.__len__()
# shuffled_indices = np.random.permutation(DATA_len)

# 使用指定的data
# shuffled_indices = np.load("../BasicInfo/arr.npy")
shuffled_indices = np.load("shuffled_indices.npy")
DATA_len = len(shuffled_indices)
np.random.shuffle(shuffled_indices)

train_idx = shuffled_indices[:int(train_pct*DATA_len)]
# train_idx = shuffled_indices
tmp = int((train_pct+vali_pct)*DATA_len)
val_idx = shuffled_indices[int(train_pct*DATA_len):tmp]

test_idx = shuffled_indices[tmp:]
print(train_idx)

[ 236    2  576  178  308    9  984   29  199  563  391  432  361  258
  617  190  648  993  271  527  843 1086  972  728  512  696  211  753
  255 1073  541  825  664  747  171 1081  579  261  968  898  282  727
 1053  316  862  811  269  674  760 1011  773  660 1098 1144  568  723
  925  987  824  585  538  575  256   42  719  392  107  518  730   35
  105 1013  684  539  782  584  758   94  287  229  819  732  381  522
 1004  103  817  919  237  835  695 1025 1032 1031 1065  153  989  640
  910  398  504  806   44  501  166 1162  717  212  347  858  822  860
  856 1147 1034  248  707  385  484  814 1136 1112  996   98  880  850
  187 1066  350  715  369  991  596  155  450 1115  126 1131 1190  480
 1148  276  802  716  962  610   65   71   16  786 1038  449  958  458
  831  408  192  520  205  928  257  310  731  189  530  889  127  164
   48  370  263  994 1195  490  916  266  407  966  233  892  131  740
 1174   93  709 1187  467  213   63  434  528  635  182  531  163   60
  491 

## 2.2 Dataloader and collating
1. 主要是对label数据进行collate
    - 按照batch中的最大target data长度进行padding，padding with 0
2. 返回的结果多一个batch dim,比如下面的`5`
    - After collating:
        - `torch.Size([5, 3, 300]),torch.Size([5, 87, 2])`
        - `87`是最长的targets data长度

In [31]:
np.save('shuffled_indices',shuffled_indices)

In [33]:
def my_collate_fn(data):
# 这里的data是一个list， list的元素是元组: (self.data, self.label)
# collate_fn的作用是把[(data, label),(data, label)...]转化成([data, data...],[label,label...])
# 假设self.data的一个data的shape为(channels, length), 每一个channel的length相等,
# data[索引到index(batch)][索引到data或者label][索引到channel]

    data_list = []
    target_list = []
    target_da_list = []
    setting_list = []
    metric_list = []

    data_len = len(data)      # 读进来的data batch的大小

    batch = 0

    while batch < data_len:
        # print("shape:",data[batch][0].shape) shape: (3, 300)

        data_list.append(torch.tensor(data[batch][0][2,:]))

        target_list.append(torch.tensor(data[batch][1]))
        target_da_list.append(torch.tensor(data[batch][2]))
        setting_list.append(torch.tensor(data[batch][3]))
        metric_list.append(torch.tensor(data[batch][4]))
        batch += 1

    # Pad target data with zeros
    target_padded = pad_sequence(target_list,batch_first=True)
    target_da_padded = pad_sequence(target_da_list,batch_first=True)
    target_tensor = target_padded.float()
    target_da_padded = target_da_padded.float()

    data_tensor = torch.stack(data_list).float()
    setting_tensor = torch.stack(setting_list).float()
    metric_tensor = torch.stack(metric_list).float()

    return data_tensor, target_tensor, target_da_padded, setting_tensor, metric_tensor

In [34]:
train_loader = DataLoader(dataset = dataset,batch_size = batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(train_idx), collate_fn = my_collate_fn)

val_loader = DataLoader(dataset = dataset,batch_size = batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(val_idx),collate_fn = my_collate_fn)

# 注意test_loader的batch size
test_loader = DataLoader(dataset = dataset,batch_size = batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(test_idx),collate_fn = my_collate_fn)

# 3. The Net and Init
1. BatchNorm1d: The mean and std are calculated per-dimension over the mini-batches
## 3.1 2层MLP

In [10]:
# 参数量32253+
# 2层
class MLP_1(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=300,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=100,affine=True)
        self.BN3 = nn.BatchNorm1d(num_features=12,affine=True)
        self.drop = nn.Dropout(0.3)

        self.linear1 = nn.Linear(300, 100)
        self.linear2 = nn.Linear(100, 12)

        self.ac_func = nn.Softplus()
        self.ac_func2 = nn.SELU()
        self.ac_func3 = nn.LeakyReLU()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(12, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(12, n_gaussians)
        self.z_sigma = nn.Linear(12, n_gaussians)

        # SELU Init
        # for m in self.modules():
        #     if isinstance(m, (nn.Linear)):
        #         nn.init.kaiming_normal_(m.weight, nonlinearity='linear')

    def forward(self, x):
        # print(x.shape)
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func3(self.drop(x))

        x = self.linear2(x)
        x = self.BN3(x)
        x = self.ac_func3(self.drop(x))

        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)
        # sigma = torch.exp(self.z_sigma(x))
        sigma = F.elu(self.z_sigma(x)) + 1
        sigma = torch.clamp(sigma,1e-4)

        # return x
        return pi, mu, sigma

## 3.2 1层MLP

In [9]:
# 参数量3417
# 1层
class MLP_2(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=300,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=9,affine=True)
        self.drop = nn.Dropout(0.3)

        self.linear1 = nn.Linear(300, 9)

        self.ac_func = nn.Softplus()
        self.ac_func2 = nn.SELU()
        self.ac_func3 = nn.LeakyReLU()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(9, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(9, n_gaussians)
        self.z_sigma = nn.Linear(9, n_gaussians)

        # SELU Init
        # for m in self.modules():
        #     if isinstance(m, (nn.Linear)):
        #         nn.init.kaiming_normal_(m.weight, nonlinearity='linear')

    def forward(self, x):
        # print(x.shape)
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func(self.drop(x))

        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)
        sigma = torch.exp(self.z_sigma(x))
        # sigma = F.elu(self.z_sigma(x)) + 1
        sigma = torch.clamp(sigma,1e-4)

        # return x
        return pi, mu, sigma

# 4. The Loss

## 4.0 The metric
1. 以NLL作为metric
2. 理论模型的metric可以提前算好，读进来，这里读setting是为了？IDK

In [12]:
# from loss import cal_metric

## 4.1 NLL loss

In [13]:
# from loss import loss_fn_v2

## 4.4 W-distance
1. 拿到target data的两列，按照第一列去predict

In [14]:
# from loss import loss_fn_WD

# 5 Plot
1. draw:
    - loss: training data的loss和test data的loss趋势
    - MLP的网络结构（.png）
    - target distrb.和pred. distrb.
    - 所有target data的分布图


In [10]:
writer = SummaryWriter("logs-MLP")
viz = Visdom(env="001")
print("Done")

Setting up a new session...


Done


## 5.2
### 5.2.1 mdn pdf的图像[with input data]
1. 求解[1,6,11...]上的cdf：
    - 求解[1,6,11..]的cdf，再求解[6,11,..]，两者相减

In [16]:
def draw_mdn_2(pi, mu, sigma, target, input, total_train_step, N_gaussians):
    """
    画两条曲线，pred-1表示归一化前，pred-2表示归一化后

    :param pi:
    :param mu:
    :param sigma:
    :param target:
    :param input:
    :param total_train_step:
    :param N_gaussians:
    :return:
    """
    # 只画一个batch中的第一个

    # The target distrb.
    target = torch.unique(target,dim=0)    # drop the duplicate
    n_target = target[:,0]
    non_zero_idx = torch.nonzero(n_target)
    n = n_target[non_zero_idx]

    p_target = target[:,1]
    p = p_target[non_zero_idx]
    max_n = int(max(n).item())

    # The predicted distrb.
    m = torch.distributions.Normal(mu,sigma)          # Gaussian
    # m = torch.distributions.Laplace(mu,sigma)           # Laplace

    x_0 = torch.arange(1,(max_n+TARGET),TARGET).to(device=device)

    x = torch.repeat_interleave(x_0.unsqueeze(dim=1), repeats=N_gaussians, dim=1)   # expand dim
    # 求解每个长度为TARGET的区间上的cdf
    y = (m.cdf(x+TARGET) - m.cdf(x)).to(device=device)

    # 方法一：
    y_pred_1 = torch.sum(pi*y,dim=1)
    # 方法二：做一下(0,1)上的归一化
    y_pred_2 = y_pred_1/y_pred_1.sum()

    # The input distrb.
    # input_data = input[0:2,:]
    input_data = input[0,:]
    x_input = torch.arange(1,(300+1),SCALE).to(device=device)
    y_input_0 = input_data.to(device=device)
    # y_input_0 = (input_data[0,:]).to(device=device)
    # y_input_1 = (input_data[1,:]).to(device=device)

    # Init
    win_str = "total_train_step = "+str(total_train_step)
    title_str = "Distrb. in "+win_str
    viz.line(X = [0.],Y = [0.], env="001", win=win_str, opts= dict(title=title_str))

    # Visdom本身不能把hist和line画在一个window中
    # 如果想画一起只能是两条line
    # Plot y_target
    # viz.histogram(X = n, env="001", win=win_str,
    #         opts= dict(title=title_str))
    viz.line(X = n,Y= p, env="001", win=win_str, update="append", name='target',
            opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

    # Plot y_pred
    viz.line(X = x_0,Y= y_pred_1, env="001", win=win_str, update="append", name='pred-1',
            opts= dict(title=title_str))

    # Plot y_pred_2
    viz.line(X = x_0,Y= y_pred_2, env="001", win=win_str, update="append", name='pred-2',
            opts= dict(title=title_str))

    # Plot y_input
    # 如果input data长度短，全部画完
    if(len(x_input) <= max_n):
        viz.line(X = x_input,Y= y_input_0, env="001", win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
        # viz.line(X = x_input,Y= y_input_1, env="001", win=win_str, update="append", name='GT-2', opts= dict(title=title_str))
    # 如果input data长度长，则截断
    else:
        x_input_0 = x_input[0:max_n]
        y_input_2 = y_input_0[0:max_n]/y_input_0[0:max_n].sum()
        # y_input_3 = y_input_1[0:max_n]/y_input_1[0:max_n].sum()
        viz.line(X = x_input_0,Y= y_input_2, env="001", win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
        # viz.line(X = x_input_0,Y= y_input_3, env="001", win=win_str, update="append", name='GT-2', opts= dict(title=title_str))

## 5.4 plot the loss
1. 直接根据参数决定draw什么loss，反正loss都画在一个window里面

In [12]:
win_train_loss_str = "The Loss of BATCH in the Training Data"
win_vali_loss_str = "The Loss in the Vali Data"
win_train_epoch_loss_str = "The Loss of EPOCH in the Training Data"

def draw_loss(X_step, loss, win_str):
    viz.line(X = [X_step], Y = [loss],win=win_str, update="append",
        opts= dict(title=win_str))


## 5.5 plot the metric

In [13]:
win_vali_metric_str = "The NLL of ALL vali data"

def draw_metric(X_step, total_vali_metric, GT_metric):

    viz.line(X = [X_step], Y = [[total_vali_metric,GT_metric[0,0],GT_metric[0,1]]],win=win_vali_metric_str, update="append", opts= dict(title=win_vali_metric_str, legend=['pred','GT-1','GT-2'], showlegend=True,xlabel="epoch", ylabel="NLL"))

# 6 Training
## 6.1 Init

In [35]:
# torch.autograd.set_detect_anomaly(True)

mlp = MLP_2(N_gaussians)

# Init the params
# # mlp = model_param_init(mlp)
# Init the vis
viz.line(X = [0.],Y = [0.], env="001", win=win_train_loss_str, opts= dict(title=win_train_loss_str))
# viz.line(X = [0.],Y = [0.], env="001", win=win_vali_loss_str, opts= dict(title=win_vali_loss_str))
viz.line(X = [0.],Y = [0.], env="001", win=win_train_epoch_loss_str, opts= dict(title=win_train_epoch_loss_str))
viz.line(X = [0.],Y = [[0.,0.,0.]], env="001", win=win_vali_metric_str, opts= dict(title=win_vali_metric_str,legend=['pred','GT-1','GT-2'], showlegend=True,xlabel="epoch", ylabel="NLL"))
# Save the init params
# torch.save(mlp.state_dict(), 'mlp_init.pth')

# Load the saved model
# model_path_WD = "mlp_train=300_WD.pth"
# model_path_LSE = "mlp_train=300_LSE.pth"
# model_data = torch.load(model_path_LSE)
# mlp.load_state_dict(model_data)

mlp = mlp.to(device=device)
summary(mlp, (300,))

############ learning rate strategy ############
# 1.
# Set different lr for params
# id: id() 函数返回对象的唯一标识符，标识符是一个整数。返回对象的内存地址。
mu_params = list(map(id, mlp.z_mu.parameters()))
base_params = filter(lambda p: id(p) not in mu_params, mlp.parameters())
params = [{'params': base_params},         # 如果对某个参数不指定学习率，就使用最外层的默认学习率
        {'params': mlp.z_mu.parameters(), 'lr': lr_for_mu}]
optimizer = torch.optim.Adam(params, lr=learning_rate)

# optimizer = torch.optim.Adagrad(params,lr=learning_rate,weight_decay = 1e-4)
# optimizer = torch.optim.Adagrad(params,lr=learning_rate)

# 2.
# Or set lr decay
# StepLR为步进，每step_size个epoch，lr*gamma
# scheduler  = torch.optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.8)
# 下面这个比较好用
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15,30], gamma=0.7, last_epoch=-1)

# # Set the hooks
# #mlp.conv1.register_forward_hook(hook_forward_fn)
# mlp.conv1.register_full_backward_hook(hook_backward_fn)
#
# #mlp.linear1.register_forward_hook(hook_forward_fn)
# #mlp.linear1.register_full_backward_hook(hook_backward_fn)
#
# #mlp.z_pi.register_forward_hook(hook_forward_fn)
# mlp.z_pi.register_full_backward_hook(hook_backward_fn_pi)
#
# #mlp.z_mu.register_forward_hook(hook_forward_fn)
# mlp.z_mu.register_full_backward_hook(hook_backward_fn_mu)
#
# mlp.z_sigma.register_forward_hook(hook_forward_fn)
# mlp.z_sigma.register_full_[2, 3, 300]backward_hook(hook_backward_fn_sigma)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                  [-1, 300]             600
           Flatten-2                  [-1, 300]               0
            Linear-3                    [-1, 9]           2,709
       BatchNorm1d-4                    [-1, 9]              18
           Dropout-5                    [-1, 9]               0
          Softplus-6                    [-1, 9]               0
            Linear-7                    [-1, 3]              30
           Softmax-8                    [-1, 3]               0
            Linear-9                    [-1, 3]              30
           Linear-10                    [-1, 3]              30
Total params: 3,417
Trainable params: 3,417
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total

## 6.3 START HERE
1. 使用Laplace分布改：
    - loss function和draw

In [36]:
# filename = "../log_file.txt"
# f = open(filename,'w')
total_train_step = 0
EPOCH_NUM = 60

train_start_time = time.time()
mlp.train()

for epoch in range(EPOCH_NUM):
    # # Save the net structure
    # net_file_name = "net_before_epoch"+str(epoch)+".pth"
    # net_path = net_root_path + net_file_name
    # torch.save(mlp.state_dict(), net_path)

    epoch_train_loss = 0
    print(f"========== Now this is EPOCH {epoch} ==========")
    for batch_id,data in enumerate(train_loader):

        input_data, _, target_loss, setting, target_metric = data
        # print(f"---- {batch_id} batch----")
        # Do the inference
        input_data = input_data.to(device)
        target_loss = target_loss.to(device)
        pi, mu, sigma = mlp(input_data)

        # Save the params
        # params = list(mlp.named_parameters())

        # Cal the MLE loss and draw the distrb.
        # loss = loss_fn_v2(pi, mu, sigma, target, N_gaussians)
        loss = loss_fn_v2(pi, mu, sigma, target_loss, N_gaussians,TARGET, SAFETY, device)
        # loss = loss_fn_v3(pi, mu, sigma, target_da, N_gaussians)
        # loss = loss_fn_v5(pi, mu, sigma, target_da, N_gaussians)

        # Cal the CDF loss and draw the distrb.
        # loss = loss_fn_cdf(pi, mu, sigma, target, N_gaussians)

        # Cal the CE or KL loss and draw the distrb.
        # loss  = loss_fn_CE(pi, mu, sigma, target, N_gaussians)

        # Cal the WD loss and draw the distrb.
        # loss  = loss_fn_WD(pi, mu, sigma, target, N_gaussians)

        epoch_train_loss += loss.item()
        draw_loss(total_train_step, loss.item(),win_train_loss_str)

        # print("训练次数：{}，Loss：{}".format(total_train_step, loss.item()))

        # Optim
        optimizer.zero_grad()
        loss.backward()

        ########### before step() ###############

        # for name, parms in mlp.named_parameters():
        #     print("=====before step()=====")
        #     print('-->name:', name)
        #     print('-->params:', parms.data)
        #     # print('-->grad_requirs:',parms.requires_grad)
        #     print('-->grad_value:',parms.grad)
        #     print("===")

        ######

        # 反向传播时检测是否有异常值，定位code
        # with torch.autograd.detect_anomaly():
        #     loss.backward()

        ######
        # get_dot = register_hooks(loss)
        # dot = get_dot()
        # dot.save('tmp.dot')   # to get .dot
        # dot.render('tmp')     # to get pdf

        ######
        # Print grad check
        # v_n = []    # name
        # v_v = []    # value
        # v_g = []    # grad
        # for name, parameter in mlp.named_parameters():
        #     v_n.append(name)
        #     v_v.append(parameter.detach().cpu().numpy() if parameter is not None else [0])
        #     v_g.append(parameter.grad.detach().cpu().numpy() if parameter.grad is not None else [0])
        # for i in range(len(v_n)):
        #     # if np.max(v_v[i]).item() - np.min(v_v[i]).item() < 1e-6:
        #     #     color = bcolors.FAIL + '*'
        #     if np.isnan(v_g[i]).any() or np.isnan(v_v[i]).any():
        #         color = bcolors.FAIL + '*'
        #     else:
        #         color = bcolors.OKGREEN + ' '
        #     print('%svalue %s: %.3e ~ %.3e' % (color, v_n[i], np.min(v_v[i]).item(), np.max(v_v[i]).item()))
        #     print('%sgrad  %s: %.3e ~ %.3e' % (color, v_n[i], np.min(v_g[i]).item(), np.max(v_g[i]).item()))

        ######
        optimizer.step()
        scheduler.step()

        # for name, parms in mlp.named_parameters():
        #     print("=====After step()=====")
        #     print('-->name:', name)
        #     print('-->params:', parms.data)
        #     # print('-->grad_requirs:',parms.requires_grad)
        #     print('-->grad_value:',parms.grad)
        #     print("===")

        # Only draw the 1st result in a training batch (5 in total)
        # if total_train_step % 20 == 0:
        #     with torch.no_grad():
        #         draw_mdn_2(pi[0,:].clone().detach(), mu[0,:].clone().detach(), sigma[0,:].clone().detach(), target[0].clone().detach(), input_data[0].clone().detach(),total_train_step, N_gaussians)
        #         # draw_mdn_cdf(pi[0,:].detach(), m[0], target[0].detach(), total_train_step, loss_list[0],N_gaussians)
        total_train_step += 1


    ########### Do validation
    with torch.no_grad():
        mlp.eval()
        total_vali_metric = 0
        GT_metric = torch.tensor([0.,0.]).reshape(1,2)

        for vali_batch_id, vali_data in enumerate(val_loader):
            vali_input_data, vali_target, _, vali_setting , vali_metric = vali_data
            vali_input_data = vali_input_data.to(device)
            vali_target = vali_target.to(device)
            # vali_metric = vali_metric

            vali_pi, vali_mu, vali_sigma = mlp(vali_input_data)

            # Compute the error/ metric
            vali_nll = cal_metric(vali_pi, vali_mu, vali_sigma, vali_target, N_gaussians, vali_setting,MIN_LOSS,device)
            total_vali_metric += vali_nll

            # Sum up NLL of all vali data
            GT_metric += torch.sum(vali_metric,dim=0)
        # total_vali_metric = total_vali_metric/len(val_idx)

        # Get metric of GT model
        GT_metric = GT_metric/len(val_idx)
        total_vali_metric = total_vali_metric/len(val_idx)

        draw_metric(epoch, total_vali_metric.cpu(), GT_metric)
        mlp.train()

    # Plot the loss in this EPOCH
    print(f"========== IN EPOCH {epoch} the total loss is {epoch_train_loss} ==========")

    # # Save the net structure
    # net_file_name = "net_after_epoch"+str(epoch)+".pth"
    # net_path = net_root_path + net_file_name
    # torch.save(mlp.state_dict(), net_path)

    # Plot loss of this EPOCH
    draw_loss(epoch, epoch_train_loss,win_train_epoch_loss_str)
    # writer.add_scalars("Loss of EPOCH",{"Train":epoch_train_loss},epoch)

    # Record the params
    # for name,param in params:
    #     #参数的梯度
    #     #print("name:",name)
    #     #print("param.grad:",param.grad)
    #     # writer.add_histogram(tag = name +'_grad of EPOCH '+str(epoch),values=param.grad,global_step=epoch)
    #     # #参数值
    #     writer.add_histogram(tag = name +'data of EPOCH '+str(epoch),values=param.data,global_step=epoch)

# f.close()
train_end_time = time.time()
print(f"Total training time when epoch= *{EPOCH_NUM}* is *{train_end_time-train_start_time} *s")
writer.close()
print("Done")

========== Now this is EPOCH 0 ==========
========== IN EPOCH 0 the total loss is 73195.24108886719 ==========
========== Now this is EPOCH 1 ==========
========== IN EPOCH 1 the total loss is 51030.161865234375 ==========
========== Now this is EPOCH 2 ==========
========== IN EPOCH 2 the total loss is 43566.20520019531 ==========
========== Now this is EPOCH 3 ==========
========== IN EPOCH 3 the total loss is 34874.15148925781 ==========
========== Now this is EPOCH 4 ==========
========== IN EPOCH 4 the total loss is 28139.33319091797 ==========
========== Now this is EPOCH 5 ==========
========== IN EPOCH 5 the total loss is 23095.932739257812 ==========
========== Now this is EPOCH 6 ==========
========== IN EPOCH 6 the total loss is 19099.593963623047 ==========
========== Now this is EPOCH 7 ==========
========== IN EPOCH 7 the total loss is 16989.895965576172 ==========
========== Now this is EPOCH 8 ==========
========== IN EPOCH 8 the total loss is 17781.479431152344 =======

In [37]:
model_path_MLE = "mlp_train=300_MLE.pth"
# model_path_WD = "mlp_train=300_WD_epoch=80.pth"
model_path_WD = "mlp_train=300_WD.pth"
model_path_LSE = "mlp_train=300_LSE.pth"
torch.save(mlp.state_dict(), model_path_LSE)
# torch.save(mlp.state_dict(), model_path_MLE)

# viz.delete_env("001_test")
